### 3.6 MNIST digit
- https://github.com/WegraLee/deep-learning-from-scratch
- 밑바닥부터 시작하는 딥러닝의 dataset를 clone하여 진행하였다.

In [1]:
from dataset.mnist import load_mnist

In [4]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

In [5]:
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


- flatten : 입력 이미지를, 1차원 배열로 만들지 여부 or 1 X 28 X 28의 3차원 배열로 만들지 여부
- normalize : 입력 이미지의 픽셀 값을 0.0 ~ 1.0 사이의 값으로 정규화 할지 여부
- one_hot_label : 레이블을 원-핫 인코딩 형태로 저장할 지 여부

In [6]:
from PIL import Image

In [20]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    # pil_img.show()
    return pil_img

In [21]:
img = x_train[0]
label = t_train[0]
print("label: {}".format(label))
print("image array shape : {}".format(img.shape))
img = img.reshape(28, 28)
print("image array reshape to 28 x 28 : {}".format(img.shape))

label: 5
image array shape : (784,)
image array reshape to 28 x 28 : (28, 28)


In [22]:
img_show(img)

- 주의사항 : flatten을 True 로 해주었기 때문에, 1차원 np array 로 저장되어 있다.
- 이를 원래 이미지로 보여주기 위해서는 28 X 28 크기로 다시 복구하여야 한다.

- numpy 1차원 Array (784, ) -> numpy 2차원 Array (28, 28) -> PIL 객체 : Image.fromarray(np.uing8(img))

### 3.6.2 신경망의 추론
- 입력층 : 28 * 28 -> 784개 뉴런
- 첫번째 은닉층 : 50개 뉴런
- 두번째 은닉층 : 100개 뉴런
- 출력층 : 10개 뉴런(0 ~ 9 분류)

In [24]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [25]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

In [28]:
import pickle
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open("./sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y
    

In [29]:
x, t = get_data()
network = init_network()

In [32]:
len(t)

10000

In [30]:
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)
    if p == t[i]:
        accuracy_cnt += 1
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


### 3.6.3 배치 처리

In [34]:
x, _ = get_data()
network = init_network()

In [35]:
W1, W2, W3 = network['W1'], network['W2'], network['W3']

In [36]:
x.shape

(10000, 784)

In [38]:
x[0].shape

(784,)

In [39]:
W1.shape

(784, 50)

In [40]:
W2.shape

(50, 100)

In [41]:
W3.shape

(100, 10)

In [42]:
x, t = get_data()
network = init_network()

batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
    
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


In [50]:
list(range(0, 11))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [51]:
list(range(0, 11, 2))

[0, 2, 4, 6, 8, 10]

In [54]:
x, t = get_data()
network = init_network()

batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    print(i, y_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
    
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

0 [[3.29137538e-06 1.02745233e-07 2.78982516e-05 4.90758575e-05
  4.57290881e-08 1.75426374e-06 6.34363728e-10 3.88771258e-02
  3.65525295e-07 3.19072205e-05]
 [2.38913308e-05 5.45662306e-06 4.66457661e-03 7.06867577e-05
  2.81526891e-09 3.29794384e-05 1.36013929e-04 6.27792041e-09
  6.35489550e-06 2.36451081e-10]
 [4.35422171e-10 4.15724982e-03 1.80313364e-05 7.49591254e-06
  5.53756195e-07 3.19225273e-06 1.97111376e-06 9.54208372e-06
  5.20351978e-06 3.64514051e-07]
 [1.26391780e-02 1.04401154e-09 2.02235296e-05 2.41281214e-06
  5.49404113e-08 4.28640742e-05 5.16990394e-06 2.94927554e-06
  6.14853832e-07 5.34488834e-07]
 [3.63746949e-07 1.15563372e-08 5.08722860e-06 5.85677675e-08
  1.68065750e-03 8.47780029e-07 3.58964280e-06 9.65849540e-06
  2.52931227e-06 5.74644182e-05]
 [1.77338061e-10 5.78273693e-03 1.01278101e-05 1.30993240e-05
  4.94069639e-07 1.74704792e-06 2.03225952e-07 2.79188262e-05
  1.11904674e-05 2.48993501e-06]
 [1.27649624e-08 1.51145599e-07 6.71478304e-08 4.8220727

2600 [[1.46394319e-08 6.01105194e-06 1.01684891e-05 3.87167202e-05
  4.02156161e-07 2.15887194e-05 3.26211733e-08 7.92464050e-07
  3.93049000e-03 2.78946527e-06]
 [3.62264814e-06 3.80181234e-07 6.65614425e-05 4.43261783e-08
  2.25940985e-05 3.53850555e-05 1.55703621e-02 9.44621945e-08
  7.62568277e-07 8.73890382e-09]
 [2.17869825e-07 3.39322251e-05 2.31030434e-02 6.34641212e-04
  4.69142947e-09 7.83087387e-07 1.52640473e-06 4.01821717e-06
  2.10384860e-06 6.27098062e-09]
 [1.06793898e-08 3.47109705e-07 1.12152236e-07 7.60024659e-06
  5.63194953e-05 5.25768837e-06 1.95126812e-08 2.55314426e-05
  4.72109277e-05 9.14679002e-03]
 [1.15614000e-06 2.88155252e-06 1.57427235e-06 2.23280498e-04
  6.19706242e-08 4.82477248e-04 1.16255387e-06 3.32706435e-07
  3.21366169e-05 2.28127774e-07]
 [1.42278327e-06 1.20738704e-07 1.81565468e-07 7.76541128e-06
  2.12977716e-06 2.53764010e-05 2.55794497e-09 3.65056396e-02
  3.58124737e-07 1.98787035e-04]
 [2.18601735e-06 1.50401959e-07 1.46419881e-08 1.1546

5300 [[2.98155136e-08 3.89624574e-06 3.33915727e-06 1.97857185e-06
  6.57112196e-07 1.49942643e-05 4.42012009e-07 4.62281697e-08
  1.34185643e-03 1.71548061e-06]
 [6.03311928e-03 5.69866432e-10 1.00654543e-05 8.18739011e-07
  1.10970518e-07 4.07046828e-05 7.05760522e-06 1.59196574e-07
  2.99518570e-06 4.18808753e-07]
 [2.57441889e-06 1.88213562e-07 1.33256535e-08 3.66280110e-05
  1.94171250e-07 5.04993461e-03 1.23140225e-07 1.92478197e-07
  3.13416422e-05 3.49555444e-06]
 [5.26663477e-08 5.65947801e-07 2.79761171e-05 1.59526820e-07
  7.73638603e-05 3.50154664e-06 4.27682465e-03 5.43997558e-08
  6.10589439e-07 3.01050065e-08]
 [1.84980948e-07 1.27143608e-06 3.39714243e-05 2.67431432e-07
  1.12143352e-05 9.65777781e-06 9.82367992e-03 7.88143684e-09
  1.04656226e-06 8.72688410e-09]
 [5.52157722e-02 2.85932611e-10 3.00686543e-06 8.46152489e-07
  2.34191209e-08 3.49801237e-04 5.11987992e-07 9.43127077e-07
  1.56584065e-06 6.05664411e-07]
 [1.09075600e-07 2.63494917e-06 2.22595190e-06 7.7506

7700 [[1.08992474e-07 2.55524100e-08 2.99945491e-08 5.62911919e-06
  6.50414313e-06 1.17410564e-05 8.66289884e-09 1.24119724e-05
  2.43620366e-06 9.68841021e-04]
 [1.19377228e-06 3.40377433e-08 3.95061143e-05 5.33798411e-05
  3.71439484e-08 2.48170664e-07 1.21885591e-09 9.13538225e-03
  4.29894307e-08 7.41769327e-06]
 [4.55030613e-03 7.06735392e-10 7.21396646e-06 1.03142656e-06
  2.40410820e-08 3.35279910e-05 1.56104340e-06 3.11822163e-07
  8.65734933e-07 2.95477406e-07]
 [4.88387048e-03 3.82572918e-10 4.92772415e-06 5.73594832e-07
  4.77674682e-08 3.47693931e-05 8.24241567e-07 3.43219369e-07
  1.82641850e-06 5.30170439e-07]
 [1.83922335e-08 3.68623461e-08 2.21747058e-07 1.64104321e-07
  1.09870429e-03 2.45839669e-06 1.26087605e-06 4.68838743e-06
  5.18072284e-07 1.62491433e-05]
 [6.37061191e-07 2.83086450e-07 5.17277003e-05 5.09372171e-08
  5.64009179e-06 1.67055032e-06 6.98474189e-03 9.51524903e-09
  1.02660135e-06 7.40512851e-09]
 [8.00174754e-03 3.30295152e-10 4.05353603e-06 8.4017

In [55]:
A = np.array([[1,2,3],
              [4,5,6],
              [7,8,9]
             ])
A

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [58]:
np.argmax(A, axis=1)

array([2, 2, 2])